## Customer Segmentation Notebook

This single-file, runnable Python notebook/script walks you through:
1. Loading dataset (Mall Customers or UCI Online Retail II)
2. EDA (basic stats & missingness)
3. RFM feature engineering (for transactional data)
4. Preprocessing & scaling
5. Clustering: KMeans (with elbow/silhouette helper) and DBSCAN
6. Dimensionality reduction for visualization (PCA + optional UMAP)
7. Cluster profiling and export (CSV) ready for Power BI
8. A simple ETL function to re-run RFM & assign clusters (for scheduling)

Instructions:
- Put your dataset CSV in `data/` folder. Filenames expected by default:
    - `data/mall_customers.csv`  (Mall Customers small dataset)
    - OR `data/online_retail.csv` (UCI Online Retail / Online Retail II export)
- Optional: create a virtualenv/conda and `pip install -r requirements.txt` with
    pandas, numpy, scikit-learn, matplotlib, seaborn, umap-learn (optional), joblib

Run this script cell-by-cell in Jupyter or as a .py script. The script tries to be robust to small format differences.

In [ ]:
# Required packages
import os
import sys
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import joblib

try:
    import umap
    _HAS_UMAP = True
except Exception:
    _HAS_UMAP = False





In [ ]:
# -----------------------------
# load data
# -----------------------------

def load_dataset(preferred='mall'):
    """Try to load mall_customers.csv or online_retail.csv from data/ directory.
    preferred: 'mall' or 'retail'
    Returns (df, dataset_name)
    """
    base = 'data'
    mall_path = os.path.join(base, 'mall_customers.csv')
    retail_path = os.path.join(base, 'online_retail.csv')

    if preferred == 'mall' and os.path.exists(mall_path):
        df = pd.read_csv(mall_path)
        return df, 'mall'
    if preferred == 'retail' and os.path.exists(retail_path):
        df = pd.read_csv(retail_path)
        return df, 'retail'

    # fallback
    if os.path.exists(mall_path):
        return pd.read_csv(mall_path), 'mall'
    if os.path.exists(retail_path):
        return pd.read_csv(retail_path), 'retail'

    raise FileNotFoundError('No dataset found. Put mall_customers.csv or online_retail.csv in the data/ folder.')

In [ ]:
# -----------------------------
# Basic EDA function
# -----------------------------

def basic_eda(df, max_rows=5):
    print('Shape:', df.shape)
    print('\nColumns and dtypes:')
    print(df.dtypes)
    print('\nNull counts:')
    print(df.isnull().sum().sort_values(ascending=False).head(20))
    print('\nSample rows:')
    display = df.head(max_rows)
    print(display)


In [ ]:

# -----------------------------
# RFM computation (for transactional retail)
# -----------------------------

def compute_rfm_from_retail(df, customer_id_col='CustomerID', invoice_date_col='InvoiceDate', invoice_no_col='InvoiceNo', amount_col='TotalAmount', snapshot_date=None):
    """
    Expects a transactional dataframe with one transaction per row.
    If your dataset does not have a TotalAmount column, create it as Quantity * UnitPrice.
    """
    df = df.copy()
    # Standardize columns if necessary
    if invoice_date_col in df.columns and not np.issubdtype(df[invoice_date_col].dtype, np.datetime64):
        df[invoice_date_col] = pd.to_datetime(df[invoice_date_col], errors='coerce')

    if amount_col not in df.columns and 'Quantity' in df.columns and 'UnitPrice' in df.columns:
        df[amount_col] = df['Quantity'] * df['UnitPrice']

    if snapshot_date is None:
        snapshot_date = df[invoice_date_col].max() + pd.Timedelta(days=1)

    # Filter out canceled rows if InvoiceNo has a 'C' (common in Online Retail)
    if invoice_no_col in df.columns:
        if df[invoice_no_col].dtype == object:
            df = df[~df[invoice_no_col].astype(str).str.contains('C', na=False)]

    # Drop rows missing customer id
    df = df.dropna(subset=[customer_id_col])

    # Aggregate R, F, M
    rfm = df.groupby(customer_id_col).agg(
        Recency=('InvoiceDate', lambda x: (snapshot_date - x.max()).days),
        Frequency=(invoice_no_col, 'nunique'),
        Monetary=(amount_col, 'sum')
    ).reset_index()

    # Handle possible zeros or negatives
    rfm['Monetary'] = rfm['Monetary'].clip(lower=0)
    return rfm



In [ ]:
# -----------------------------
# For Mall Customers dataset: minimal features
# -----------------------------

def preprocess_mall(df):
    # Expected columns: CustomerID (optional), Age, Annual Income (k$), Spending Score (1-100), Gender
    df = df.copy()
    # Rename common variants
    cols = {c: c.strip() for c in df.columns}
    df.columns = list(cols.keys())

    # If CustomerID missing, create one
    if 'CustomerID' not in df.columns:
        df['CustomerID'] = range(1, len(df)+1)

    # Select a sane subset
    features = []
    for cand in ['Age', 'Annual Income (k$)', 'Annual Income', 'Spending Score (1-100)', 'Spending Score']:
        if cand in df.columns:
            features.append(cand)
    if 'Gender' in df.columns:
        features.append('Gender')

    if not features:
        # fallback: use numeric columns
        features = df.select_dtypes(include=[np.number]).columns.tolist()

    proc = df[['CustomerID'] + features].copy()
    # Basic cleaning
    proc = proc.dropna(subset=['CustomerID'])
    return proc



In [ ]:


# -----------------------------
# Scaling & clustering helpers
# -----------------------------

def scale_features(X, scaler=None):
    if scaler is None:
        scaler = StandardScaler()
        Xs = scaler.fit_transform(X)
        return Xs, scaler
    else:
        return scaler.transform(X), scaler


def find_best_k(X, k_min=2, k_max=10):
    """Compute inertia (elbow) and silhouette for k in range."""
    results = []
    for k in range(k_min, k_max+1):
        km = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = km.fit_predict(X)
        sil = silhouette_score(X, labels)
        results.append({'k':k, 'inertia': km.inertia_, 'silhouette': sil})
    return pd.DataFrame(results)


def run_kmeans(X, k):
    km = KMeans(n_clusters=k, random_state=42, n_init=20)
    labels = km.fit_predict(X)
    return km, labels


def run_dbscan(X, eps=0.5, min_samples=5):
    db = DBSCAN(eps=eps, min_samples=min_samples)
    labels = db.fit_predict(X)
    return db, labels



In [ ]:


# -----------------------------
# Visualization helpers
# -----------------------------

def plot_elbow_silhouette(df_res):
    fig, ax1 = plt.subplots(1,2, figsize=(12,4))
    ax1[0].plot(df_res['k'], df_res['inertia'], marker='o')
    ax1[0].set_xlabel('k'); ax1[0].set_title('Inertia (Elbow)')
    ax1[1].plot(df_res['k'], df_res['silhouette'], marker='o')
    ax1[1].set_xlabel('k'); ax1[1].set_title('Silhouette Score')
    plt.tight_layout()
    plt.show()


def plot_clusters_2d(X, labels, title='Clusters (2D PCA)'):
    pca = PCA(n_components=2, random_state=42)
    comp = pca.fit_transform(X)
    dfp = pd.DataFrame(comp, columns=['PC1','PC2'])
    dfp['label'] = labels
    plt.figure(figsize=(8,6))
    sns.scatterplot(data=dfp, x='PC1', y='PC2', hue='label', palette='tab10', s=40)
    plt.title(title)
    plt.legend(bbox_to_anchor=(1.05,1), loc='upper left')
    plt.tight_layout()
    plt.show()


def umap_plot(X, labels):
    if not _HAS_UMAP:
        print('UMAP not installed. Install with: pip install umap-learn')
        return
    reducer = umap.UMAP(random_state=42)
    embed = reducer.fit_transform(X)
    dfp = pd.DataFrame(embed, columns=['UMAP1','UMAP2'])
    dfp['label'] = labels
    plt.figure(figsize=(8,6))
    sns.scatterplot(data=dfp, x='UMAP1', y='UMAP2', hue='label', palette='tab10', s=30)
    plt.title('UMAP projection')
    plt.legend(bbox_to_anchor=(1.05,1), loc='upper left')
    plt.tight_layout()
    plt.show()



In [ ]:


# -----------------------------
# Cluster profiling & export
# -----------------------------

def profile_and_export(df_customers, features, labels, id_col='CustomerID', out_csv='outputs/customer_clusters.csv'):
    df = df_customers.copy()
    df['cluster'] = labels
    # For numeric features show cluster means
    prof = df.groupby('cluster')[features].agg(['mean','median','count']).round(3)
    print('Cluster profile (numeric features):')
    print(prof)

    os.makedirs(os.path.dirname(out_csv), exist_ok=True)
    df.to_csv(out_csv, index=False)
    print('Saved cluster assignments to', out_csv)
    return prof



In [ ]:

# -----------------------------
# End-to-end runner for Mall dataset
# -----------------------------

def run_segmentation_mall():
    df_raw, name = load_dataset('mall')
    print('Loaded dataset:', name)
    mall = preprocess_mall(df_raw)
    basic_eda(mall)

    # Choose features
    # If Gender present, convert to numeric
    proc = mall.copy()
    if 'Gender' in proc.columns:
        proc['Gender'] = proc['Gender'].astype(str).str.lower().map({'male':0,'female':1}).fillna(-1)

    # find numeric features except CustomerID
    features = [c for c in proc.columns if c != 'CustomerID']
    X = proc[features].select_dtypes(include=[np.number]).fillna(0)

    Xs, scaler = scale_features(X)

    # Determine k
    dfk = find_best_k(Xs, k_min=2, k_max=8)
    print(dfk)
    plot_elbow_silhouette(dfk)

    # pick k (you can change)
    k = int(dfk.sort_values('silhouette', ascending=False).iloc[0]['k'])
    print('Choosing k =', k)
    km, labels = run_kmeans(Xs, k=k)

    plot_clusters_2d(Xs, labels, title=f'Mall Customers - KMeans k={k}')
    prof = profile_and_export(proc[['CustomerID'] + features], features, labels)
    joblib.dump({'scaler':scaler, 'kmeans':km, 'features':features}, 'outputs/mall_segmentation_pipeline.joblib')
    print('Saved pipeline to outputs/mall_segmentation_pipeline.joblib')



In [ ]:

# -----------------------------
# End-to-end runner for Retail dataset (RFM)
# -----------------------------

def run_segmentation_retail():
    df_raw, name = load_dataset('retail')
    print('Loaded dataset:', name)
    basic_eda(df_raw)

    # Attempt to create TotalAmount if not present
    if 'TotalAmount' not in df_raw.columns and 'Quantity' in df_raw.columns and 'UnitPrice' in df_raw.columns:
        df_raw['TotalAmount'] = df_raw['Quantity'] * df_raw['UnitPrice']

    rfm = compute_rfm_from_retail(df_raw, customer_id_col='CustomerID', invoice_date_col='InvoiceDate', invoice_no_col='InvoiceNo', amount_col='TotalAmount')
    print('RFM head:')
    print(rfm.head())

    # log transform Monetary to reduce skew
    rfm['Monetary_log'] = np.log1p(rfm['Monetary'])
    features = ['Recency','Frequency','Monetary_log']
    X = rfm[features]
    Xs, scaler = scale_features(X)

    # find k
    dfk = find_best_k(Xs, k_min=2, k_max=8)
    print(dfk)
    plot_elbow_silhouette(dfk)

    # choose best silhouette
    k = int(dfk.sort_values('silhouette', ascending=False).iloc[0]['k'])
    print('Choosing k =', k)
    km, labels = run_kmeans(Xs, k=k)

    plot_clusters_2d(Xs, labels, title=f'Retail RFM - KMeans k={k}')
    prof = profile_and_export(rfm[['CustomerID'] + features], features, labels, id_col='CustomerID', out_csv='outputs/retail_customer_clusters.csv')
    joblib.dump({'scaler':scaler, 'kmeans':km, 'features':features}, 'outputs/retail_segmentation_pipeline.joblib')
    print('Saved pipeline to outputs/retail_segmentation_pipeline.joblib')



In [ ]:

# -----------------------------
# Simple ETL wrapper to recompute clusters (callable from CLI)
# -----------------------------

def run_pipeline(dataset='mall'):
    if dataset == 'mall':
        run_segmentation_mall()
    elif dataset == 'retail':
        run_segmentation_retail()
    else:
        raise ValueError('dataset must be mall or retail')



In [ ]:


# -----------------------------
# If executed as script, try to run both if data present
# -----------------------------
if __name__ == '__main__':
    try:
        # try retail first, then mall
        if os.path.exists('data/online_retail.csv'):
            run_pipeline('retail')
        elif os.path.exists('data/mall_customers.csv'):
            run_pipeline('mall')
        else:
            print('No dataset found in data/. Put mall_customers.csv or online_retail.csv there and re-run.')
    except Exception as e:
        print('Error during run:', e)
        raise